In [1]:
from pyspark.sql import SparkSession

# создаём сессию Spark
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .getOrCreate()
)

In [2]:
import os
import pyspark.sql.functions as sql_func

# читаем файл с оценками (26 млн)
DATA_DIR = "/data/ml-latest"
ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "ratings.csv"),
        header=True,
        inferSchema=True
    )
    .drop("timestamp")
)

In [3]:
# разбиваем на train и test
train, test = ratings.randomSplit([0.8, 0.2], seed=42)
train.cache()
test.cache()

DataFrame[userId: int, movieId: int, rating: double]

In [4]:
%%time
from pyspark.ml.recommendation import ALS

# обучаем модель с базовыми настройками
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    # если в тестовой выборке будет встречаться неизвестный фильм
    # или неизвестный пользователь, они будут игнорироваться
    coldStartStrategy="drop",
    seed=42
).fit(train)

CPU times: user 70 ms, sys: 30 ms, total: 100 ms
Wall time: 1min 42s


In [5]:
%%time
from pyspark.ml.evaluation import RegressionEvaluator

# оцениваем RMSE
predictions = als.transform(test)
rmse = evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
).evaluate(predictions)
print("RMSE на тестовой выборке:", rmse)

RMSE на тестовой выборке: 0.8155114007283507
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 38.4 s
